In [19]:
import glob 
from sklearn.utils import shuffle
import re
import urduhack
import pandas as pd
from urduhack.normalization import normalize
from urduhack.preprocessing import normalize_whitespace, remove_punctuation, remove_accents, replace_urls, replace_emails, replace_currency_symbols
from typing import FrozenSet
import stanfordnlp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import f1_score
from feature_selection import FeatureSelection
from sklearn.feature_selection import SelectKBest
from functools import partial

### 1. Import training and test data for Urdu fake news classification

In [32]:
train_news = []
train_label = []

files = glob.glob("../datasets/fake-new-corpus/Train/Real/*.txt", recursive = True) 

for each in files:
    file = open(each, encoding = "UTF-8")
    text = file.read()
    train_news.append(text)
    train_label.append(0)
    
files = glob.glob("../datasets/fake-new-corpus/Train/Fake/*.txt", recursive = True) 

for each in files:
    file = open(each, encoding = "UTF-8")
    text = file.read()
    train_news.append(text)
    train_label.append(1)
    
# conver to dataframe
training_df = pd.DataFrame({'text': train_news, 'label': train_label})

#shuffle the dataframe
training_df = shuffle(training_df)

In [33]:
test_news = []
test_label = []

files = glob.glob("../datasets/fake-new-corpus/Test/Real/*.txt", recursive = True) 

for each in files:
    file = open(each, encoding = "UTF-8")
    text = file.read()
    test_news.append(text)
    test_label.append(0)
    
files = glob.glob("../datasets/fake-new-corpus/Test/Fake/*.txt", recursive = True) 

for each in files:
    file = open(each, encoding = "UTF-8")
    text = file.read()
    test_news.append(text)
    test_label.append(1)

# store to a dataframe

test_df = pd.DataFrame({'text': test_news, 'label': test_label})

### 2. Data Preprocessing

In [34]:
def preprocess(text):
    # remove the html tags
    text = re.sub('<.*?>', '', text) 

    # remove punctuation marks and numbers
    #text = re.sub(r"[^a-zA-Z@]+", ' ', text)
    text = re.sub("\d+", " ", text)

    # remove URLs
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    
    # remove new line character
    text = text.replace('\n', ' ')
    text = text.replace('،', ' ')
    
    # remove English stopwords
    stop_words = ['AFP', 'Image', 'caption', 'Reuters', 'EPA', 'AFPGetty', 'getty', 
                  'Getty', 'Images', 'AFP/Getty Images', 'AFP/Getty', 'Sunday', 'Monday', 'Wednesday', 'July']
    
    words = text.split(' ')
    
    final_text = []
    
    for word in words: 
        if word:
            if not word in stop_words:
                final_text.append(word)
    
    return " ".join(final_text)

In [35]:
# apply urduhack normalization on training/test data
training_df.text = training_df.text.apply(normalize).apply(remove_punctuation).apply(remove_accents).apply(replace_urls).apply(replace_emails).apply(replace_currency_symbols).apply(normalize_whitespace)
test_df.text = test_df.text.apply(normalize).apply(remove_punctuation).apply(remove_accents).apply(replace_urls).apply(replace_emails).apply(replace_currency_symbols).apply(normalize_whitespace)

In [36]:
# define Urdu stopwords
STOP_WORDS: FrozenSet[str] = frozenset("""
 آ آئی آئیں آئے آتا آتی آتے آس آمدید آنا آنسہ آنی آنے آپ آگے آہ آہا آیا اب ابھی ابے
 ارے اس اسکا اسکی اسکے اسی اسے اف افوہ البتہ الف ان اندر انکا انکی انکے انہوں انہی انہیں اوئے اور اوپر
 اوہو اپ اپنا اپنوں اپنی اپنے اپنےآپ اکثر اگر اگرچہ اہاہا ایسا ایسی ایسے ایک بائیں بار بارے بالکل باوجود باہر
 بج بجے بخیر بشرطیکہ بعد بعض بغیر بلکہ بن بنا بناؤ بند بڑی بھر بھریں بھی بہت بہتر تاکہ تاہم تب تجھ
 تجھی تجھے ترا تری تلک تم تمام تمہارا تمہاروں تمہاری تمہارے تمہیں تو تک تھا تھی تھیں تھے تیرا تیری تیرے
 جا جاؤ جائیں جائے جاتا جاتی جاتے جانی جانے جب جبکہ جدھر جس جسے جن جناب جنہوں جنہیں جو جہاں جی جیسا
 جیسوں جیسی جیسے حالانکہ حالاں حصہ حضرت خاطر خالی خواہ خوب خود دائیں درمیان دریں دو دوران دوسرا دوسروں دوسری دوں
 دکھائیں دی دیئے دیا دیتا دیتی دیتے دیر دینا دینی دینے دیکھو دیں دیے دے ذریعے رکھا رکھتا رکھتی رکھتے رکھنا رکھنی
 رکھنے رکھو رکھی رکھے رہ رہا رہتا رہتی رہتے رہنا رہنی رہنے رہو رہی رہیں رہے ساتھ سامنے ساڑھے سب سبھی
 سراسر سمیت سوا سوائے سکا سکتا سکتے سہ سہی سی سے شاید شکریہ صاحب صاحبہ صرف ضرور طرح طرف طور علاوہ عین
 فقط فلاں فی قبل قطا لئے لائی لائے لاتا لاتی لاتے لانا لانی لانے لایا لو لوجی لوگوں لگ لگا لگتا
 لگتی لگی لگیں لگے لہذا لی لیا لیتا لیتی لیتے لیکن لیں لیے لے ماسوا مت مجھ مجھی مجھے محترم محترمہ محض
 مرا مرحبا مری مرے مزید مس مسز مسٹر مطابق مل مکرمی مگر مگھر مہربانی میرا میروں میری میرے میں نا نزدیک
 نما نہ نہیں نیز نیچے نے و وار واسطے واقعی والا والوں والی والے واہ وجہ ورنہ وغیرہ ولے وگرنہ وہ وہاں
 وہی وہیں ویسا ویسے ویں پاس پایا پر پس پلیز پون پونی پونے پھر پہ پہلا پہلی پہلے پیر پیچھے چاہئے
 چاہتے چاہیئے چاہے چلا چلو چلیں چلے چناچہ چند چونکہ چکی چکیں چکے ڈالنا ڈالنی ڈالنے ڈالے کئے کا کاش کب کبھی
 کدھر کر کرتا کرتی کرتے کرم کرنا کرنے کرو کریں کرے کس کسی کسے کم کن کنہیں کو کوئی کون کونسا
 کونسے کچھ کہ کہا کہاں کہہ کہی کہیں کہے کی کیا کیسا کیسے کیونکر کیونکہ کیوں کیے کے گئی گئے گا گنا
 گو گویا گی گیا ہائیں ہائے ہاں ہر ہرچند ہرگز ہم ہمارا ہماری ہمارے ہمی ہمیں ہو ہوئی ہوئیں ہوئے ہوا
 ہوبہو ہوتا ہوتی ہوتیں ہوتے ہونا ہونگے ہونی ہونے ہوں ہی ہیلو ہیں ہے یا یات یعنی یک یہ یہاں یہی یہیں
      """.split())

def remove_stopwords(text):
    return " ".join(word for word in text.split() if word not in STOP_WORDS)

In [37]:
def remove_special_char(text):
    return text.replace("\ufeff", "")

In [38]:
# remove stopwords and any special characters
training_df.text = training_df.text.apply(preprocess)
training_df.text = training_df.text.apply(remove_stopwords)   
training_df.text = training_df.text.apply(remove_special_char) 


test_df.text = test_df.text.apply(preprocess)
test_df.text = test_df.text.apply(remove_stopwords)   
test_df.text = test_df.text.apply(remove_special_char)

In [39]:
training_df.head()

,text,label
217,اڈیلیڈ ڈسمبر سیاست ڈاٹ کام ہندوستانی کرکٹ ٹیم ...,0
115,آنکھیں توجہ طالب آنکھ نازک عضو صحت خاص احتیاط ...,0
596,پاکستان درآمد بطور تحائف دئیے ٹیلی فونز بلاک ج...,1
210,cnnurdu شائع نئی دہلی بھارتی ویمن ٹی ٹوئنٹی کر...,0
243,تصویر کاپی رائٹ سرفراز احمد رنز آؤٹ ہوگئے انڈی...,0


### 3. Tokenization and binary vectorization

In [40]:
# configure the stanford nlp tokenizer for urdu
config = {
    'processors': 'tokenize', # Comma-separated list of processors to use
    'lang': 'ur', # Language code for the language to build the Pipeline in
    'tokenize_model_path': r'D:\stanfordnlp_resources\ur_udtb_models\ur_udtb_tokenizer.pt'
}

nlp = stanfordnlp.Pipeline(**config)

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'D:\\stanfordnlp_resources\\ur_udtb_models\\ur_udtb_tokenizer.pt', 'lang': 'ur', 'shorthand': 'ur_udtb', 'mode': 'predict'}
Done loading processors!
---


In [41]:
# define the tokenizer function
def tokenizer(text):
    doc = nlp(text)
    tokens = []
    for i, sentence in enumerate(doc.sentences):
        tokens += [f"{token.text}" for token in sentence.tokens]

    return tokens

In [42]:
# convert the data to binary vectors
vectorizer = CountVectorizer(binary=True, tokenizer=tokenizer)

vectorized_train_x = vectorizer.fit_transform(training_df.text)

In [51]:
vectorized_train_x.shape

(638, 12496)

In [43]:
# vectorize the test set as well
vectorized_test_x = vectorizer.transform(test_df.text)

### 4. Feature ranking metrics

In [52]:
# select features based on different feature ranking metrics
fs = FeatureSelection()

# the results can be reproduced with different feature selection measure and different value of k
selector = SelectKBest(score_func=partial(fs.IG, vectorizer = vectorizer), k=1500)
new_train_x = selector.fit_transform(vectorized_train_x, training_df.label)

In [53]:
new_test_x = selector.transform(vectorized_test_x)

In [54]:
clf = BernoulliNB()
clf.fit(new_train_x, training_df.label)

y_hat = clf.predict(new_test_x.toarray())
f1 = f1_score(test_df.label, y_hat)

print(f1)

0.6789667896678966
